# FIU DATA

In [ ]:
import numpy as np
import os

path = "./data/"
fiu_list = []

for file in os.listdir(path):
    if file.startswith("fiu"):
        fiu_list.append(np.load(path+file))

In [ ]:
fiu_array = np.concatenate(fiu_list, 0)

In [ ]:
np.save("./fiu_data.npy", fiu_array)

# Typeface Dict

In [ ]:
import numpy as np
import os
import urllib.request

path = "./data/"
type_list = []

for file in os.listdir(path):
    if file.startswith("typeface"):
        type_list.append(np.load(path+file).item())

In [ ]:
new_type_dict = {}
for dic in type_list:
    for (key, value) in dic.items():
        if new_type_dict.get(key) == None:
            new_type_dict[key] = value
        else:
            continue

In [ ]:
import urllib.request
dw_path = "./data/png/"

for (name, url) in new_type_dict.items():
    name = name.replace(' ','_').replace('/','-')
    urllib.request.urlretrieve(url, dw_path+name+".png")

# FastText

In [8]:
import io
fin = io.open("../data/word_emb/wiki-news-300d-1M.vec", 
              'r', encoding='utf-8', newline='\n', errors='ignore')
n, d = map(int, fin.readline().split())

In [9]:
ft_dict = {'u*n*k':0, 'p*a*d':1}
ft_list = ['u*n*k', 'p*a*d']

word2vec = {}
for line in fin:
    tokens = line.rstrip().split(' ')
    word2vec[tokens[0]] = list(map(float, tokens[1:]))
    ft_dict[tokens[0]] = len(ft_list)
    ft_list.append(tokens[0])

np.save('ft_w2v.npy', word2vec)
np.save('ft_w2i.npy', ft_dict)
np.save('ft_i2w.npy', ft_list)

In [3]:
ft_list = np.load('../data/word_emb/ft_i2w.npy')
word2vec = np.load('../data/word_emb/ft_w2v.npy').item()
print(len(ft_list), ft_list[0], ft_list[1])

999996 u*n*k p*a*d


In [12]:
ft_i2v = []
for word in ft_list[2:]:
    ft_i2v.append(word2vec[word])
ft_i2v = np.array(ft_i2v)
np.save("../data/word_emb/ft_i2v.npy", ft_i2v)

In [14]:
ft_i2v.shape

(999994, 300)

# Each Typo for an Indexed Text

In [34]:
import nltk
#nltk.download('punkt')
import numpy as np
data_arr = np.load("../data/fiu_data.npy")
ft_dict = np.load("../data/word_emb/ft_w2i.npy").item()

In [35]:
UNK = 0
PAD = 1

new_data = []
for (typo_list, text) in data_arr:
    sent_text = nltk.sent_tokenize(text.lower())
    text = [[word for word in nltk.word_tokenize(sentence)] for sentence in sent_text]
    text = [a for b in text for a in b]
    
    indexed_text = []
    for word in text:
        if ft_dict.get(word) != None:
            indexed_text.append(ft_dict[word])
        else:
            indexed_text.append(UNK)
    
    # cutting till 300 words
    if len(indexed_text) > 300:
        sent_len = 300
        indexed_text = indexed_text[:300]
    elif len(indexed_text) < 300:
        sent_len = len(indexed_text)
        indexed_text += [PAD]*(300-len(indexed_text))

    for typo in typo_list:
        new_data.append([typo] + indexed_text + [sent_len])
        
np.save('fiu_indexed.npy', new_data)

# Statistics

In [ ]:
text_len = []
for (typo, text) in new_data:
    text_len.append(len(text))
print(np.max(text_len), np.mean(text_len), np.median(text_len), np.min(text_len))
print(len(new_data))